In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

working_dir = Path.cwd()
while working_dir.name != 'CausalMTR-BC':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'CausalMTR-BC' not found")
os.chdir(working_dir)

# Load env variables from config/.env 
dotenv_path = Path("configs/.env")
load_dotenv(dotenv_path=dotenv_path)

GPUDRIVE_PATH = os.environ.get("GPUDRIVE_PATH")
sys.path.append(os.path.abspath(GPUDRIVE_PATH))

from cmtr_bc.waymo_dataset import WaymoDataset
import yaml 
from easydict import EasyDict
import logging
import torch
import numpy as np

In [2]:
def getconfig(cfg_file):
    with open(cfg_file, 'r') as f:
        try:
            config = yaml.load(f, Loader=yaml.FullLoader)
        except:
            config = yaml.load(f)

    return EasyDict(config)

logger = logging.getLogger(__name__)

config = getconfig("configs/config.yaml")
waymo_dataset = WaymoDataset(config.DATA_CONFIG, test_mode=1, logger=logger)

In [3]:
batch_size = 1
batch_list = [waymo_dataset.create_scene_level_data(i) for i in range(batch_size)]
batch_dict = waymo_dataset.collate_batch(batch_list)

In [4]:
batch_dict

{'batch_size': 1,
 'index': array([0]),
 'input_dict': {'scenario_id': array(['101d77f96894896a', '101d77f96894896a', '101d77f96894896a'],
        dtype='<U16'),
  'obj_trajs': tensor([[[[-4.4490e+00, -1.1915e-01,  1.0809e-02,  ...,  2.1448e-01,
              1.0622e+00, -8.1241e-02],
            [-4.0455e+00, -9.8511e-02,  4.4670e-03,  ...,  2.0636e-01,
              1.0622e+00, -8.1241e-02],
            [-3.6290e+00, -7.6199e-02,  3.9768e-03,  ...,  2.2312e-01,
              1.2953e+00,  1.6761e-01],
            ...,
            [-9.3871e-01,  1.8018e-02, -1.0155e-02,  ...,  3.3117e-02,
              1.0977e+00, -8.8617e-01],
            [-4.6563e-01,  1.3543e-02, -1.0921e-02,  ..., -4.4748e-02,
              6.5145e-01, -7.7865e-01],
            [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.3543e-01,
             -7.4586e-01, -9.0686e-01]],
  
           [[-1.5276e+01,  2.4994e+00, -1.5957e-01,  ...,  8.1758e-02,
              1.3575e+00,  8.7804e-01],
            [-1.4893e+01,  

In [5]:
from mtr.models import model as model_utils

/opt/miniforge3/envs/gpudrive/lib/python3.11/site-packages/colt5_attention/triton_coor_descent.py:314: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/opt/miniforge3/envs/gpudrive/lib/python3.11/site-packages/colt5_attention/triton_coor_descent.py:410: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [6]:
model = model_utils.MotionTransformer(config=config.MODEL).to("cuda:0")

In [7]:
next(model.parameters()).device

device(type='cuda', index=0)

In [8]:
# Initialize the epoch
model.set_epoch(0)

In [9]:
output = model(batch_dict)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (40514x9 and 13x64)

In [11]:
output

(tensor(64941.3516, device='cuda:0', grad_fn=<AddBackward0>),
 {'loss_layer0': 34980.7578125,
  'loss_layer0_reg_gmm': 34689.640625,
  'loss_layer0_reg_vel': 279.0565490722656,
  'loss_layer0_cls': 4.0200018882751465,
  'loss_layer1': 32516.2578125,
  'loss_layer1_reg_gmm': 32224.83984375,
  'loss_layer1_reg_vel': 279.24853515625,
  'loss_layer1_cls': 4.057023525238037,
  'loss_layer2': 31276.15234375,
  'loss_layer2_reg_gmm': 30984.3671875,
  'loss_layer2_reg_vel': 279.2383728027344,
  'loss_layer2_cls': 4.181699752807617,
  'loss_layer3': 33568.640625,
  'loss_layer3_reg_gmm': 33276.1484375,
  'loss_layer3_reg_vel': 279.74859619140625,
  'loss_layer3_cls': 4.247533321380615,
  'loss_layer4': 32464.880859375,
  'loss_layer4_reg_gmm': 32171.544921875,
  'loss_layer4_reg_vel': 280.253173828125,
  'loss_layer4_cls': 4.360513210296631,
  'loss_layer5': 37088.74609375,
  'loss_layer5_reg_gmm': 36795.96875,
  'loss_layer5_reg_vel': 279.2662353515625,
  'loss_layer5_cls': 4.503271102905273,


In [12]:
len(output)

4

In [13]:
loss, tb_dict, disp_dict, batch_dict = output

In [14]:
batch_dict

{'batch_size': 1,
 'index': array([0]),
 'input_dict': {'scenario_id': array(['101d77f96894896a', '101d77f96894896a', '101d77f96894896a'],
        dtype='<U16'),
  'obj_trajs': tensor([[[[-4.4490e+00, -1.1915e-01,  1.0809e-02,  ...,  2.1448e-01,
              1.0622e+00, -8.1241e-02],
            [-4.0455e+00, -9.8511e-02,  4.4670e-03,  ...,  2.0636e-01,
              1.0622e+00, -8.1241e-02],
            [-3.6290e+00, -7.6199e-02,  3.9768e-03,  ...,  2.2312e-01,
              1.2953e+00,  1.6761e-01],
            ...,
            [-9.3871e-01,  1.8018e-02, -1.0155e-02,  ...,  3.3117e-02,
              1.0977e+00, -8.8617e-01],
            [-4.6563e-01,  1.3543e-02, -1.0921e-02,  ..., -4.4748e-02,
              6.5145e-01, -7.7865e-01],
            [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.3543e-01,
             -7.4586e-01, -9.0686e-01]],
  
           [[-1.5276e+01,  2.4994e+00, -1.5957e-01,  ...,  8.1758e-02,
              1.3575e+00,  8.7804e-01],
            [-1.4893e+01,  

In [15]:
batch_dict.keys()

dict_keys(['batch_size', 'index', 'input_dict', 'batch_sample_count', 'input_grad_dict', 'ego2obj_attn_mask', 'center_objects_feature', 'obj_feature', 'map_feature', 'obj_mask', 'map_mask', 'obj_pos', 'map_pos', 'forward_ret_dict', 'pred_scores', 'pred_trajs'])

In [18]:
pred_trajs = batch_dict['pred_trajs']
pred_trajs.shape

torch.Size([3, 6, 80, 7])

In [19]:
batch_dict['batch_size']

1

In [22]:
input_dict = batch_dict['input_dict']
input_dict.keys()

dict_keys(['scenario_id', 'obj_trajs', 'obj_trajs_mask', 'track_index_to_predict', 'obj_trajs_pos', 'obj_trajs_last_pos', 'obj_types', 'obj_ids', 'center_objects_world', 'center_objects_id', 'center_objects_type', 'obj_trajs_future_state', 'obj_trajs_future_mask', 'center_gt_trajs', 'center_gt_trajs_mask', 'center_gt_final_valid_idx', 'center_gt_trajs_src', 'map_polylines', 'map_polylines_mask', 'map_polylines_center'])

In [24]:
input_dict['track_index_to_predict'].shape

torch.Size([3])

In [ ]:
'''
Some info about pred_trajs: 

The shape: (# of ego_agents | this is the same as tracks to predict, 
            # of selected modes, 
            # of time steps to predict,
            # of features to predict)
'''

In [30]:
batch_dict['pred_trajs'].shape

torch.Size([3, 6, 80, 7])

In [ ]:
import mtr

In [2]:
print("stiff")

stiff
